In [4]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

## Simple self-attention

In [24]:
# simple self-attention mechanism
context_size = inputs.shape[0]
a_scores = torch.empty((context_size, context_size))

# 1st step is to compute the attention scores. Bellow we calculate them for all inputs.
# This kinda calculates how the embeddings of a token relate to the other tokens in the same context 
a_scores = inputs @ inputs.T

# then we apply softmax to normalize into the attention weight
attention = torch.softmax(a_scores, dim=1)

context_vector = attention @ inputs

context_vector

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

## Self-attention with trainable weights

In [27]:
# Self-attention with trainable weights
x_2 = inputs[1]
d_in = inputs.shape[1]

d_out = 2

In [36]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
W_keys = torch.nn.Parameter(torch.rand(d_in, d_out))
W_values = torch.nn.Parameter(torch.rand(d_in, d_out))

In [37]:
query_2 = x_2 @ W_query
keys = inputs @ W_keys
values = inputs @ W_values

In [ ]:
d_k = keys.shape[1]

att_score = keys @ query_2
att_score = torch.softmax(att_score / d_k**0.5, dim=-1)
att = att_score @ values
att

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
       grad_fn=<SoftmaxBackward0>)

## Creating a class for a Self-atention

In [65]:
class SelfAttention_v1(torch.nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_queries = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_keys = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_values = torch.nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        queries = x @ self.W_queries
        keys = x @ self.W_keys
        values = x @ self.W_values

        att_scores = queries @ keys.T
        att_scores = torch.softmax(att_scores / d_out**0.5, dim=-1)

        context_vector = att_scores @ values

        return context_vector
    

class SelfAttention_v2(torch.nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_queries = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_keys = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_values = torch.nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        queries = self.W_queries(x)
        keys = self.W_keys(x)
        values = self.W_values(x)

        att_scores = queries @ keys.T
        att_scores = torch.softmax(att_scores / d_out**0.5, dim=-1)

        context_vector = att_scores @ values

        return context_vector

In [67]:
torch.manual_seed(42)

d_in = inputs.shape[1]
d_out = 2

self_att_1 = SelfAttention_v1(d_in, d_out)
context = self_att_1.forward(inputs)
context

self_att_2 = SelfAttention_v2(d_in, d_out)
context = self_att_2.forward(inputs)
context

tensor([[0.1712, 0.1184],
        [0.1750, 0.1207],
        [0.1751, 0.1208],
        [0.1751, 0.1209],
        [0.1764, 0.1215],
        [0.1743, 0.1205]], grad_fn=<MmBackward0>)

## Causal attention mask

In [ ]:
import torch

# create an artificial batch
batch = torch.stack((inputs, inputs), dim=0)
batch.shape

torch.Size([2, 6, 3])

In [28]:
import torch

class CausalSelfAttention_v1(torch.nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout=0.0, qkv_bias=False):
        super().__init__()
        self.W_queries = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_keys = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_values = torch.nn.Linear(d_in, d_out, bias=qkv_bias)

        # causal mask
        self.register_buffer("causal_mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

        # dropout mask
        self.dropout = torch.nn.Dropout(dropout)

        

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        queries = self.W_queries(x)
        keys = self.W_keys(x)
        values = self.W_values(x)

        att_scores = queries @ keys.transpose(1, 2)
        att_scores.masked_fill_(
            self.causal_mask.bool()[:num_tokens, :num_tokens], -torch.inf
        )

        att_scores = torch.softmax(att_scores / d_in**0.5, dim=-1)

        att_scores = self.dropout(att_scores)

        context_vector = att_scores @ values
        return context_vector

In [27]:
torch.manual_seed(123)

d_in = inputs.shape[1]
d_out = 2

csl_self_att = CausalSelfAttention_v1(d_in, d_out, context_length=inputs.shape[0])
context = csl_self_att.forward(batch)
context

tensor([[[-0.4519,  0.2216],
         [-0.5866,  0.0071],
         [-0.6293, -0.0621],
         [-0.5670, -0.0838],
         [-0.5519, -0.0979],
         [-0.5295, -0.1077]],

        [[-0.4519,  0.2216],
         [-0.5866,  0.0071],
         [-0.6293, -0.0621],
         [-0.5670, -0.0838],
         [-0.5519, -0.0979],
         [-0.5295, -0.1077]]], grad_fn=<UnsafeViewBackward0>)

## Multi Head Attention

In [32]:
import torch

class MultiHeadAttentionWrapper(torch.nn.Module):
    def __init__(self, num_heads, d_in, d_out, context_length, dropout=0.0, qkv_bias=False):
        super().__init__()
        self.heads = torch.nn.ModuleList([
            CausalSelfAttention_v1(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads)
        ])        

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)
    

torch.manual_seed(123)

d_in = inputs.shape[1]
d_out = 2

multihead = MultiHeadAttentionWrapper(2, d_in, d_out, context_length=inputs.shape[0])
context = multihead.forward(batch)
context

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5866,  0.0071,  0.5869,  0.3214],
         [-0.6293, -0.0621,  0.6184,  0.3825],
         [-0.5670, -0.0838,  0.5474,  0.3575],
         [-0.5519, -0.0979,  0.5319,  0.3423],
         [-0.5295, -0.1077,  0.5074,  0.3481]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5866,  0.0071,  0.5869,  0.3214],
         [-0.6293, -0.0621,  0.6184,  0.3825],
         [-0.5670, -0.0838,  0.5474,  0.3575],
         [-0.5519, -0.0979,  0.5319,  0.3423],
         [-0.5295, -0.1077,  0.5074,  0.3481]]], grad_fn=<CatBackward0>)

In [ ]:
# Multi Head optimized (removing the sequential calls to the causal self attention)

class MultiHeadAttention(torch.nn.Module):
    def __init__(self, num_heads, d_in, d_out, context_length, dropout=0.0, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.d_head = d_out // num_heads

        self.W_queries = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_keys = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_values = torch.nn.Linear(d_in, d_out, bias=qkv_bias)

        self.out_proj = torch.nn.Linear(d_out, d_out)

        # causal mask
        self.register_buffer("causal_mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

        # dropout mask
        self.dropout = torch.nn.Dropout(dropout)


    def forward(self, x):
        b, num_tokens, d_in = x.shape

        # we split the matrices by adding a new dimension (d_head)
        queries = self.W_queries(x)
        queries = queries.view(b, num_tokens, self.num_heads, self.d_head)
        queries = queries.transpose(1, 2)

        keys = self.W_keys(x)
        keys = keys.view(b, num_tokens, self.num_heads, self.d_head)
        keys = keys.transpose(1, 2)

        values = self.W_values(x)
        values = values.view(b, num_tokens, self.num_heads, self.d_head)
        values = values.transpose(1, 2)

        att_scores = queries @ keys.transpose(2, 3)
        att_scores.masked_fill_(
            self.causal_mask.bool()[:num_tokens, :num_tokens], -torch.inf
        )

        att_scores = torch.softmax(att_scores / keys.shape[-1]**0.5, dim=-1)

        att_scores = self.dropout(att_scores)

        context_vector = (att_scores @ values).transpose(1,2) 

        # Combine the results from each head
        context_vector = context_vector.contiguous().view(b, num_tokens, d_out) # "equivalent" to reshape()
        context_vector = self.out_proj(context_vector)
        return context_vector

In [36]:
torch.manual_seed(123)

d_in = inputs.shape[1]
d_out = 4

multihead = MultiHeadAttention(2, d_in, d_out, context_length=inputs.shape[0])
context = multihead.forward(batch)
context

tensor([[[ 0.1184,  0.3120, -0.0847, -0.5774],
         [ 0.0178,  0.3221, -0.0763, -0.4225],
         [-0.0147,  0.3259, -0.0734, -0.3721],
         [-0.0116,  0.3138, -0.0708, -0.3624],
         [-0.0117,  0.2973, -0.0698, -0.3543],
         [-0.0132,  0.2990, -0.0689, -0.3490]],

        [[ 0.1184,  0.3120, -0.0847, -0.5774],
         [ 0.0178,  0.3221, -0.0763, -0.4225],
         [-0.0147,  0.3259, -0.0734, -0.3721],
         [-0.0116,  0.3138, -0.0708, -0.3624],
         [-0.0117,  0.2973, -0.0698, -0.3543],
         [-0.0132,  0.2990, -0.0689, -0.3490]]], grad_fn=<ViewBackward0>)